# Brainhack ASR (STT En Fast Conformer-Transducer Large)


# 1. Load libraries and dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%pip install Cython jiwer librosa nemo_text_processing pyloudnorm wandb
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117
%pip install nemo_toolkit['all']

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.3/161.3 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 64.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 116.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.5 MB/s eta 0:00:00
  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 731.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 114.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling to

In [3]:
import os
import csv
import copy
import json
import math
import wandb
import librosa
import pandas as pd
import soundfile as sf
from pyloudnorm import Meter

# Model Training
import nemo.collections.asr as nemo_asr
# NeMo's Experiment manager to handle checkpoint saving and logging
from nemo.utils import exp_manager
from omegaconf import OmegaConf, open_dict
from pytorch_lightning import Trainer, seed_everything

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio

# Model Evaluation
from jiwer import wer

# Reproducibility
seed_everything(42, workers=True)

[NeMo W 2023-05-28 05:10:45 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-05-28 05:10:48 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

Load Dataset

In [4]:
if not os.path.exists("asr_data/"):
  os.makedirs("asr_data")

In [13]:
# Unzip files in the train set into the asr_data folder
!unzip '/content/drive/MyDrive/Hackathon/Brainhack_ASR/BrainHack TIL-23 Automatic Speech Recognition Advanced Hackathon/Train.zip' -d asr_data

Archive:  /content/drive/MyDrive/Hackathon/Brainhack_ASR/BrainHack TIL-23 Automatic Speech Recognition Advanced Hackathon/Train.zip
   creating: asr_data/Train/
   creating: asr_data/Train/audio/
  inflating: asr_data/Train/audio/train_02752.wav  
  inflating: asr_data/Train/audio/train_01140.wav  
  inflating: asr_data/Train/audio/train_01913.wav  
  inflating: asr_data/Train/audio/train_03404.wav  
  inflating: asr_data/Train/audio/train_03074.wav  
  inflating: asr_data/Train/audio/train_02524.wav  
  inflating: asr_data/Train/audio/train_02514.wav  
  inflating: asr_data/Train/audio/train_02989.wav  
  inflating: asr_data/Train/audio/train_01117.wav  
  inflating: asr_data/Train/audio/train_01203.wav  
  inflating: asr_data/Train/audio/train_00834.wav  
  inflating: asr_data/Train/audio/train_02999.wav  
  inflating: asr_data/Train/audio/train_00512.wav  
  inflating: asr_data/Train/audio/train_01303.wav  
  inflating: asr_data/Train/audio/train_01126.wav  
  inflating: asr_data/Tr

In [14]:
# Unzip files in the test set into the asr_data folder
!unzip '/content/drive/MyDrive/Hackathon/Brainhack_ASR/BrainHack TIL-23 Automatic Speech Recognition Advanced Hackathon/Test_Advanced.zip' -d asr_data

Streaming output truncated to the last 5000 lines.
  inflating: asr_data/Test_Advanced/audio/evala_00331.wav  
  inflating: asr_data/Test_Advanced/audio/evala_07934.wav  
  inflating: asr_data/Test_Advanced/audio/evala_07493.wav  
  inflating: asr_data/Test_Advanced/audio/evala_04322.wav  
  inflating: asr_data/Test_Advanced/audio/evala_06078.wav  
  inflating: asr_data/Test_Advanced/audio/evala_10164.wav  
  inflating: asr_data/Test_Advanced/audio/evala_10471.wav  
  inflating: asr_data/Test_Advanced/audio/evala_03296.wav  
  inflating: asr_data/Test_Advanced/audio/evala_11727.wav  
  inflating: asr_data/Test_Advanced/audio/evala_07784.wav  
  inflating: asr_data/Test_Advanced/audio/evala_01264.wav  
  inflating: asr_data/Test_Advanced/audio/evala_07636.wav  
  inflating: asr_data/Test_Advanced/audio/evala_01064.wav  
  inflating: asr_data/Test_Advanced/audio/evala_04603.wav  
  inflating: asr_data/Test_Advanced/audio/evala_07993.wav  
  inflating: asr_data/Test_Advanced/audio/evala_0

In [7]:
brainhack_ds_filepath = '/content/drive/MyDrive/Hackathon/Brainhack_ASR/BrainHack TIL-23 Automatic Speech Recognition Advanced Hackathon'

In [8]:
# Define path to dataset and to save model
DATASET_TRAIN = f'{brainhack_ds_filepath}/Train.csv'
FILE_TEST = f'{brainhack_ds_filepath}/SampleSubmission_Advanced.csv'
TRAIN_DATA_DIR = '/content/asr_data/Train/'
TEST_DATA_DIR = '/content/asr_data/Test_Advanced/'

In [9]:
# Load dataset
dataset = pd.read_csv(DATASET_TRAIN)
# Shuffle the records
dataset = dataset.sample(frac=1, random_state=1).reset_index(drop=True)
# Load file for storing model predictions
test_set = pd.read_csv(FILE_TEST)

In [10]:
dataset["annotation"] = dataset["annotation"].str.lower() # normalize to lower case

In [11]:
# Define train test split
TRAIN_TEST_SPLIT = 0.8

# Split the dataset
train_proportion = int(TRAIN_TEST_SPLIT * len(dataset))
train_set = dataset[:train_proportion]
dev_set = dataset[train_proportion:]

In [15]:
# Build manifest files
def build_manifest(ds, manifest_path, is_train_set):
  with open(manifest_path, 'w') as mp:
    for idx, record in ds.iterrows():
      # Get audio file
      audio_path = record['path']

      if is_train_set:
        # Get duration
        duration = librosa.core.get_duration(filename=f'{TRAIN_DATA_DIR}{audio_path}')
        # Get transcript
        transcript = record['annotation']
        metadata = {
          "audio_filepath": f'{TRAIN_DATA_DIR}{audio_path}',
          "duration": duration,
          "text": transcript
        }
      else:
        # Get duration 
        duration = librosa.core.get_duration(filename=f'{TEST_DATA_DIR}audio/{audio_path}')
        metadata = {
          "audio_filepath": f'{TEST_DATA_DIR}audio/{audio_path}',
          "duration": duration
        } 

      json.dump(metadata, mp)
      mp.write('\n')

# Create manifests file if it hasn't been created
train_manifest_file = os.path.join(TRAIN_DATA_DIR, 'train_manifest.json')
if not os.path.isfile(train_manifest_file):
  build_manifest(train_set, train_manifest_file, is_train_set=True)

val_manifest_file = os.path.join(TRAIN_DATA_DIR, 'val_manifest.json')
if not os.path.isfile(val_manifest_file):
  build_manifest(dev_set, val_manifest_file, is_train_set=True)

test_manifest_file = os.path.join(TEST_DATA_DIR, 'test_manifest.json')
if not os.path.isfile(test_manifest_file):
  build_manifest(test_set, test_manifest_file, is_train_set=False)

print('***Manifests created***')

[NeMo W 2023-05-28 05:24:23 nemo_logging:349] <ipython-input-15-2ac6fbc03313>:10: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
    	This alias will be removed in version 1.0.
      duration = librosa.core.get_duration(filename=f'{TRAIN_DATA_DIR}{audio_path}')
    
[NeMo W 2023-05-28 05:26:19 nemo_logging:349] <ipython-input-15-2ac6fbc03313>:20: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
    	This alias will be removed in version 1.0.
      duration = librosa.core.get_duration(filename=f'{TEST_DATA_DIR}audio/{audio_path}')
    


***Manifests created***


# 2. Prepare Conformer Transducer Model

## Prepare Tokenizer

In [16]:
# Prepare directory for nemo scripts
if not os.path.exists("scripts/"):
  os.makedirs("scripts")

In [17]:
# Leverage modern encoder architecture through a tokenizer based model
if not os.path.exists("scripts/process_asr_text_tokenizer.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tokenizers/process_asr_text_tokenizer.py

--2023-05-28 05:33:46--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tokenizers/process_asr_text_tokenizer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13860 (14K) [text/plain]
Saving to: ‘scripts/process_asr_text_tokenizer.py’

process_asr_text_to 100%[===================>]  13.54K  --.-KB/s    in 0s      

2023-05-28 05:33:46 (68.4 MB/s) - ‘scripts/process_asr_text_tokenizer.py’ saved [13860/13860]



In [18]:
VOCAB_SIZE = 32000
TOKENIZER_TYPE = "spe"

if not os.path.exists("tokenizers"):
  os.makedirs("tokenizers")

!python scripts/process_asr_text_tokenizer.py \
   --manifest=$train_manifest_file \
   --data_root="tokenizers" \
   --tokenizer=$TOKENIZER_TYPE \
   --log \
   --vocab_size=$VOCAB_SIZE

[NeMo W 2023-05-28 05:33:50 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
INFO:root:Finished extracting manifest : /content/asr_data/Train/train_manifest.json
INFO:root:Finished extracting all manifests ! Number of sentences : 3000
[NeMo I 2023-05-28 05:33:50 sentencepiece_tokenizer:315] Processing tokenizers/text_corpus/document.txt and store at tokenizers/tokenizer_spe_bpe_v32000
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokenizers/text_corpus/document.txt --model_prefix=tokenizers/tokenizer_spe_bpe_v32000/tokenizer --vocab_size=32000 --shuffle_input_sentence=true --hard_vocab_limit=false --model_type=bpe --character_coverage=1.0 --bos_id=-1 --eos_id=-1 --normalization_rule_name=nmt_nfkc_cf
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokenizers/text_corpus/document.txt
  input_format: 
  model_prefix: tokenizers/tokenizer_spe_bpe_v32000/tokenizer
  model_type: BPE
  vocab_siz

In [19]:
# Tokenizer path
TOKENIZER = os.path.join("tokenizers", f"tokenizer_spe_bpe_v32000")
TOKENIZER_TYPE_CFG = "bpe"

## Load Pre-trained Model

In [20]:
asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name='stt_en_fastconformer_transducer_large')

[NeMo I 2023-05-28 05:33:53 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_fastconformer_transducer_large/versions/1.0.0/files/stt_en_fastconformer_transducer_large.nemo to /root/.cache/torch/NeMo/NeMo_1.18.1/stt_en_fastconformer_transducer_large/74b4e49a2b22465da737dd5345767e61/stt_en_fastconformer_transducer_large.nemo
[NeMo I 2023-05-28 05:34:03 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-05-28 05:34:04 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-05-28 05:34:05 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2023-05-28 05:34:05 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    num_workers: 8
    pin_m

[NeMo I 2023-05-28 05:34:05 features:287] PADDING: 0


[NeMo W 2023-05-28 05:34:06 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2023-05-28 05:34:06 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-05-28 05:34:15 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.18.1/stt_en_fastconformer_transducer_large/74b4e49a2b22465da737dd5345767e61/stt_en_fastconformer_transducer_large.nemo.


In [21]:
# Parameters
FREEZE_ENCODER = True
AUGMENT_AUDIO = False
EPOCHS = 10

## Freeze Encoder
Since the TIL dataset is quite small, it is better to freeze the encoder

In [22]:
def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

In [23]:
if FREEZE_ENCODER:
    asr_model.encoder.freeze()
    asr_model.encoder.apply(enable_bn_se)
    print("Model encoder has been frozen")
else:
    asr_model.encoder.unfreeze()
    print("Model encoder has been un-frozen")

Model encoder has been frozen


## Configure Model

In [24]:
cfg = copy.deepcopy(asr_model.cfg)

Set up tokenizer

In [25]:
cfg.tokenizer.dir = TOKENIZER
cfg.tokenizer.type = TOKENIZER_TYPE_CFG

# Set tokenizer config
asr_model.cfg.tokenizer = cfg.tokenizer

In [26]:
print(OmegaConf.to_yaml(cfg.tokenizer))

dir: tokenizers/tokenizer_spe_bpe_v32000
type: bpe
model_path: nemo:e7bbc43ca90943c989b60aece82c16c8_tokenizer.model
vocab_path: nemo:b7966c4186cd467b8e612a7d9384f273_vocab.txt
spe_tokenizer_vocab: nemo:0e7783691b714849aa760378530c1f39_tokenizer.vocab



Set up data loaders

In [27]:
# Setup train, validation, test configs
with open_dict(cfg):    
  # Train dataset
  cfg.train_ds.manifest_filepath = train_manifest_file
  cfg.train_ds.normalize_transcripts = False
  cfg.train_ds.batch_size = 32
  cfg.train_ds.num_workers = 8
  cfg.train_ds.pin_memory = True
  cfg.train_ds.trim_silence = True

  # Validation dataset
  cfg.validation_ds.manifest_filepath = val_manifest_file
  cfg.validation_ds.normalize_transcripts = False
  cfg.validation_ds.batch_size = 8
  cfg.validation_ds.num_workers = 8
  cfg.validation_ds.pin_memory = True
  cfg.validation_ds.trim_silence = True

  # Test dataset
  cfg.test_ds.manifest_filepath = test_manifest_file
  cfg.test_ds.normalize_transcripts = False
  cfg.test_ds.batch_size = 8
  cfg.test_ds.num_workers = 8
  cfg.test_ds.pin_memory = True
  cfg.test_ds.trim_silence = True

In [28]:
# Setup data loaders with new configs
asr_model.setup_training_data(cfg.train_ds)
asr_model.setup_validation_data(cfg.validation_ds)
asr_model.setup_test_data(cfg.test_ds)

[NeMo I 2023-05-28 05:34:16 collections:193] Dataset loaded with 3000 files totalling 3.31 hours
[NeMo I 2023-05-28 05:34:16 collections:194] 0 files were filtered totalling 0.00 hours


[NeMo W 2023-05-28 05:34:16 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    


[NeMo I 2023-05-28 05:34:17 collections:193] Dataset loaded with 750 files totalling 0.82 hours
[NeMo I 2023-05-28 05:34:17 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-05-28 05:34:17 collections:193] Dataset loaded with 12000 files totalling 13.26 hours
[NeMo I 2023-05-28 05:34:17 collections:194] 0 files were filtered totalling 0.00 hours


Set up optimisation and augmentation for the dataset to converge to a reasonable score within a short training run

In [29]:
with open_dict(asr_model.cfg.optim):
  asr_model.cfg.optim.lr = 5e-3 # from model config
  asr_model.cfg.optim.betas = [0.9, 0.98]  # from model config
  asr_model.cfg.optim.weight_decay = 1e-3  # Original weight decay
  asr_model.cfg.optim.sched.warmup_steps = None  # Remove default number of steps of warmup
  asr_model.cfg.optim.sched.warmup_ratio = 0.05  # 5 % warmup
  asr_model.cfg.optim.sched.min_lr = 5e-4  # from model config

In [30]:
# Original optimizer + scheduler
print(OmegaConf.to_yaml(asr_model.cfg.optim))

name: adamw
lr: 0.005
betas:
- 0.9
- 0.98
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: null
  warmup_ratio: 0.05
  min_lr: 0.0005



In [31]:
asr_model.cfg.joint.fuse_loss_wer = True
asr_model.cfg.joint.fused_batch_size = 16

In [32]:
asr_model.cfg.compute_eval_loss = True

Set up Data Augmentation

In [33]:
if AUGMENT_AUDIO:
    with open_dict(asr_model.cfg.spec_augment):
      asr_model.cfg.spec_augment.freq_masks = 2
      asr_model.cfg.spec_augment.freq_width = 25
      asr_model.cfg.spec_augment.time_masks = 2
      asr_model.cfg.spec_augment.time_width = 0.05
else:
    with open_dict(asr_model.cfg.spec_augment):
      asr_model.cfg.spec_augment.freq_masks = 0
      asr_model.cfg.spec_augment.time_masks = 0

In [34]:
print(OmegaConf.to_yaml(asr_model.cfg.spec_augment))

_target_: nemo.collections.asr.modules.SpectrogramAugmentation
freq_masks: 0
time_masks: 0
freq_width: 27
time_width: 0.05



# 3. Model Training
Setup Trainer and Experiment Manager

In [35]:
if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

# Initialize a Trainer for the Transducer model
trainer = Trainer(devices=1,
                  accelerator=accelerator,
                  max_epochs=EPOCHS,
                  enable_checkpointing=False,
                  logger=False,
                  log_every_n_steps=5,
                  check_val_every_n_epoch=1,
                  deterministic=True)

# Setup model with the trainer
asr_model.set_trainer(trainer)

# Update model's internal configuration
asr_model.cfg = asr_model._cfg

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [36]:
!wandb login 23c74d0b24c250fc7fe6df6c59e5ee5bca3340cd

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [37]:
%env WANDB_PROJECT=TIL2023-ASR
%env WANDB_NOTEBOOK_NAME=asr_lazythink.ipynb

env: WANDB_PROJECT=TIL2023-ASR
env: WANDB_NOTEBOOK_NAME=asr_lazythink.ipynb


In [38]:
# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/',
    name=f"nvidia-stt-ft-02-spe",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
    create_wandb_logger=True,
    wandb_logger_kwargs={
        "project" : "TIL2023-ASR",
        "name": "nvidia-stt-ft-02-spe"
    }
)

exp_config = OmegaConf.structured(exp_config)

logdir = exp_manager.exp_manager(trainer, exp_config)

[NeMo I 2023-05-28 05:35:33 exp_manager:374] Experiments will be logged at experiments/nvidia-stt-ft-02-spe/2023-05-28_05-35-33
[NeMo I 2023-05-28 05:35:33 exp_manager:797] TensorboardLogger has been set up
[NeMo I 2023-05-28 05:35:33 exp_manager:812] WandBLogger has been set up


In [39]:
%%time
trainer.fit(asr_model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-05-28 05:35:36 modelPT:722] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        eps: 1e-08
        foreach: None
        lr: 0.005
        maximize: False
        weight_decay: 0.001
    )
[NeMo I 2023-05-28 05:35:36 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f7e38404580>" 
    will be used during training (effective maximum steps = 940) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.05
    min_lr: 0.0005
    max_steps: 940
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoder                  | 108 M 
2 | decoder           | RNNTDecoder                       | 3.9 M 
3 | joint             | RNNTJoint                         | 1.4 M 
4 | loss              | RNNTLoss                          | 0     
5 | spec_augmentation | SpectrogramAugmentation           | 0     
6 | wer               | RNNTBPEWER                        | 0     
------------------------------------------------------------------------
5.4 M     Trainable params
108 M     Non-trainable params
114 M     Total params
456.382   Total estimated model params size (MB)
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find asr_lazythink.ipynb.
wandb: Currently logged in as: ti

Sanity Checking: 0it [00:00, ?it/s]

[NeMo W 2023-05-28 05:35:51 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    


...

[NeMo I 2023-05-28 06:03:28 nemo_model_checkpoint:168] New best .nemo model saved to: /content/experiments/nvidia-stt-ft-02-spe/2023-05-28_05-35-33/checkpoints/nvidia-stt-ft-02-spe.nemo


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


[NeMo I 2023-05-28 06:04:09 nemo_model_checkpoint:168] New best .nemo model saved to: /content/experiments/nvidia-stt-ft-02-spe/2023-05-28_05-35-33/checkpoints/nvidia-stt-ft-02-spe.nemo


INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/experiments/nvidia-stt-ft-02-spe/2023-05-28_05-35-33/checkpoints/nvidia-stt-ft-02-spe--val_wer=0.0058-epoch=9.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/experiments/nvidia-stt-ft-02-spe/2023-05-28_05-35-33/checkpoints/nvidia-stt-ft-02-spe--val_wer=0.0058-epoch=9.ckpt


CPU times: user 12min 57s, sys: 1min 33s, total: 14min 31s
Wall time: 29min 17s


# 4. Model Evaluation

In [40]:
# Evaluate model on the validation set
trainer.test(asr_model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

Streaming output truncated to the last 5000 lines.
[NeMo I 2023-05-28 06:06:18 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-28 06:06:18 rnnt_wer_bpe:396] predicted :they visited six different countries on their backpacking trip
[NeMo I 2023-05-28 06:06:18 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-28 06:06:18 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-28 06:06:18 rnnt_wer_bpe:396] predicted :they were lucky to escape the building with zero injuries
[NeMo I 2023-05-28 06:06:19 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-28 06:06:19 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-28 06:06:19 rnnt_wer_bpe:396] predicted :she counted to five before taking a deep breath and calming down
[NeMo I 2023-05-28 06:06:19 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-28 06:06:19 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-28 06:06:19 rnnt_wer_bpe:396] predicted :the plant had grown four inches since she last measured it
[NeMo I 2023-05-28 06:06:19 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-28 06:06:19 rnnt_wer

[{'global_step': 940.0, 'test_wer': inf}]

# 5. Model Inferencing

In [41]:
# Convert manifest files to json format
def manifest_tojson(manifest_file: str) -> str:
    manifest_list = []
    with open(manifest_file, "r") as file:
        for line in file:
            record = json.loads(line)
            manifest_list.append(record)
    return manifest_list

def normalize_to_til(transcript: str) -> str:
    # TIL output is purely uppercase alphabet and space
    # so we normalize the output to that
    result = "".join([c.upper() if c.isalpha() else " " for c in transcript])
    # Remove double spaces
    while "  " in result:
        result = result.replace("  ", " ")
    return result

def asr_predict(manifest_file, start_idx, end_idx):
    predicted_transcripts = []
    paths = []
    count = 0
    for record in manifest_file[start_idx:end_idx]:
        transcript, metadata = asr_model.transcribe([record['audio_filepath']])
        pred = normalize_to_til(transcript[0])
        # Attempt to increase loudness if unable to transcribe
        if pred == "":
            count +=1
            path = record['audio_filepath']
            data, rate = sf.read(path)
            meter = Meter(rate)
            # Measure the loudness
            loudness = meter.integrated_loudness(data)
            print("Loudness:", loudness)
            for target_loudness in (-24, -14, -11, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0):
                print("Trying target:", target_loudness)
                # calculate the gain needed to normalize
                gain = target_loudness - loudness
                # apply gain correction
                normalized_data = data * 10**(gain/20.0)
                # write out the normalized audio
                sf.write("normalized_data.wav", normalized_data, rate)
                transcript, metadata = asr_model.transcribe(["normalized_data.wav"])
                pred = normalize_to_til(transcript[0])
                if pred != "":
                    break
            print("After normalisation:", pred)
        predicted_transcripts.append(pred)
        paths.append(record['audio_filepath'].split('/')[-1])
    print("Number of audio files too soft:", count)

    return predicted_transcripts, paths

TEST_MANIFEST = manifest_tojson(test_manifest_file)

In [42]:
BATCH_SIZE = 100
start_idx = 0
end_idx = BATCH_SIZE
iterations = math.ceil(len(TEST_MANIFEST) / BATCH_SIZE)
all_transcripts = []
all_paths = []
asr_model.to("cuda")
for batch in range(iterations):
    print(f'Batch {batch+1}: Records {start_idx} to {end_idx}')
    predicted_transcripts, paths = asr_predict(TEST_MANIFEST, start_idx, end_idx)
    all_transcripts.extend(predicted_transcripts)
    all_paths.extend(paths)
    start_idx += BATCH_SIZE
    end_idx += BATCH_SIZE

Batch 1: Records 0 to 100
Number of audio files too soft: 0
Batch 2: Records 100 to 200
Number of audio files too soft: 0
Batch 3: Records 200 to 300
Number of audio files too soft: 0
Batch 4: Records 300 to 400
Number of audio files too soft: 0
Batch 5: Records 400 to 500
Number of audio files too soft: 0
Batch 6: Records 500 to 600
Number of audio files too soft: 0
Batch 7: Records 600 to 700
Number of audio files too soft: 0
Batch 8: Records 700 to 800
Number of audio files too soft: 0
Batch 9: Records 800 to 900
Loudness: -38.100571150829346
Trying target: -24
After normalisation: THEY DECIDED TO ADOPT TWO KITTENS FROM THE ANIMAL SHELTER
Number of audio files too soft: 1
Batch 10: Records 900 to 1000
Number of audio files too soft: 0
Batch 11: Records 1000 to 1100
Number of audio files too soft: 0
Batch 12: Records 1100 to 1200
Number of audio files too soft: 0
Batch 13: Records 1200 to 1300
Number of audio files too soft: 0
Batch 14: Records 1300 to 1400
Number of audio files too 

In [43]:
def create_csv(paths, predictions, output_filename):
    # Ensure that paths and predictions are of the same length
    assert len(paths) == len(predictions), "Lists must have the same length"

    with open(output_filename, 'w', newline='') as csvfile:
        fieldnames = ['path', 'annotation']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for path, prediction in zip(paths, predictions):
            writer.writerow({'path': path, 'annotation': prediction})
create_csv(all_paths, all_transcripts, 'eval_submission_nvidia_conformer_transducer_ft_02_normalised.csv')